<a href="https://colab.research.google.com/github/rahultheogre/IPBABYOP/blob/main/BYOP_Group_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### About Dataset

##### Source
https://www.kaggle.com/datasets/gustavomodelli/waitlist-kidney-brazil

##### Context
Predicting waiting time for a deceased donor kidney transplant can help patients and clinicians to discuss management and contribute to a more efficient use of resources

##### Content
A model was developed with this data and published in PlosOne. We expected to share the data some improvements in the model that could help physicians and patients.

##### Reference: 
Sapiertein Silva JF, Ferreira GF, Perosa M, Nga HS, de Andrade LGM. A machine learning prediction model for waiting time to kidney transplant. PLoS One. 2021 May 20;16(5):e0252069. doi: 10.1371/journal.pone.0252069. PMID: 34015020. (https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0252069)

##### Acknowledgements
We would like to tanks the Secreataria de Saude do Estado de Sao Paulo to provide the data

##### Inspiration
Predict the Deceased donor transplantation using the predictors available

##### Allocation criteria
Allocation was performed as established by the National Transplantation System of the Brazilian Ministry of Health. For deceased donor transplants, allocation criteria are based on HLA matching (highest number of points for HLA DR, followed by HLA B and HLA A), recipient's age (<18 years), date of registration on the waiting list, and panel reactive antibody (PRA). A point score system based on blood group and HLA match is used as follows:

DR: 0 MM = 10 points; 1 MM = 5 points; 2 MM = 0 point;
B: 0 MM = 4 points; 1 MM = 2 points; 2 MM = 0 point;
A: 0 MM = 1 point; 1 MM = 0.5 point; 2 MM = 0 point.

Waiting time, allosensitization (cPRA >50), diabetes mellitus, and age < 18 years served as tiebreakers.

## 1. EXPLORATORY DATA ANALYSIS

1.   Load and audit the data
2.   Data preparation and tranformation
     a. Impute the missing values
     b. Outliers or extreme values
     c. Inconsistent values
3.   Data Visualization
4.   Data Analysis
     a. Uni-Variate Analysis (Measures of CT, Measures of Disp)
     b. Bi-Variate (correlation, chi-square test)
     c. Multi-variate (if needed)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [5]:
data = pd.read_csv('https://raw.githubusercontent.com/rahultheogre/IPBABYOP/main/waitlist.csv', encoding='latin-1')

In [6]:
pd.set_option('display.max_columns', None)
data.sample(15)

,Id,date,age_at_list_registration,age_cat,time_on_Dialysis,race,sex,underline_disease,diabetes,Blood_type,number_transfusion,gestation,number_gestation,prior_transplant,number_prior_transplant,subregion,cPRA,cPRA_cat,HLA_A1,HLA_A2,HLA_B1,HLA_B2,HLA_DR1,HLA_DR2,DR_00,B_00,A_00,calculated_frequency_DR.f1,calculated_frequency_DR.f2,calculated_frequency_DR.f,calculated_frequency_B.f1,calculated_frequency_B.f2,calculated_frequency_B.f,calculated_frequency_A.f1,calculated_frequency_A.f2,calculated_frequency_A.f,chagas,anti.HBc,anti.HCV,agHBs,patient_still_on_list,date_acutal,death,Time_death,Transplant,Transplant_Y_N,X36MthsTx,Time_Tx,priorization,removed_list,razon_removed,time,event
26670,32846,2009-11-09,46,18.a.60,5.0,Negra,F,HAS,1,O,0,Sim,1.0,Não,0,UNIFESP,28,Entre_0_50,30,0,8,42,3,13,heterozigoto,heterozigoto,homozigoto,0.18,0.25,4.50,0.08,0.04,0.32,0.13,0.00,0.00,Não,Não,Não,Não,Não,43307,Não,106,Não,Não,0,18.17,Não,Sim,Removido (suspenso > 365 dias),545,3
12435,16425,2014-09-12,71,Maior.60,22.0,Branca,M,Diabetes,0,A,1,Não,NaN,Não,0,HCFMUSP,0,Zero,24,26,35,44,3,4,heterozigoto,heterozigoto,heterozigoto,0.18,0.23,4.14,0.21,0.20,4.20,0.16,0.06,0.96,Não,Não,Não,Não,Não,43307,Sim,9,Óbito Lista,Não,0,8.77,Não,Não,NaN,263,2
6584,8673,2007-02-13,60,Maior.60,2.0,Negra,M,HAS,1,A,0,Não,NaN,Não,0,HCFMUSP,7,Entre_0_50,1,0,39,57,10,15,heterozigoto,heterozigoto,homozigoto,0.05,0.21,1.05,0.06,0.06,0.36,0.17,0.00,0.00,Não,Não,Não,Não,Não,43307,Sim,3,Óbito Lista,Não,0,2.87,Não,Não,NaN,86,2
7213,9522,2003-10-21,46,18.a.60,18.0,Branca,M,Outras,1,O,0,Não,NaN,Não,0,FUNDERP,0,Zero,2,68,49,52,3,4,heterozigoto,heterozigoto,heterozigoto,0.18,0.23,4.14,0.05,0.04,0.20,0.42,0.13,5.46,Não,Não,Não,Não,Não,43307,Sim,57,Óbito Lista,Não,0,56.63,Não,Não,NaN,1699,2
46322,54870,2017-06-29,63,Maior.60,10.0,Negra,F,Outras,1,O,0,Sim,3.0,Não,0,UNIFESP,0,Zero,3,29,35,44,3,4,heterozigoto,heterozigoto,heterozigoto,0.18,0.23,4.14,0.21,0.20,4.20,0.18,0.08,1.44,Não,Não,Não,Não,Sim,43307,Não,13,Não,Não,0,13.07,Não,Não,NaN,392,0
30506,37175,2007-10-09,31,18.a.60,11.0,Branca,F,HAS,1,O,1,Não,0.0,Não,0,FUNDERP,0,Zero,29,36,40,53,4,9,heterozigoto,heterozigoto,heterozigoto,0.23,0.04,0.92,0.09,0.06,0.54,0.08,0.02,0.16,Não,Não,Não,Não,Não,43307,Não,131,Sim,Sim,1,19.97,Não,Não,NaN,599,1
29317,35803,2012-05-29,34,18.a.60,3.0,Parda,F,GNC,1,A,1,Sim,3.0,Não,0,UNIFESP,0,Zero,24,30,15,39,4,15,heterozigoto,heterozigoto,heterozigoto,0.23,0.21,4.83,0.19,0.06,1.14,0.16,0.13,2.08,Não,Não,Não,Não,Sim,43307,Não,75,Não,Não,0,74.97,Não,Não,NaN,2249,0
47038,55685,2008-12-10,59,18.a.60,13.0,Branca,M,HAS,1,O,0,Não,NaN,Não,0,UNIFESP,0,Zero,3,26,7,38,4,13,heterozigoto,heterozigoto,heterozigoto,0.23,0.25,5.75,0.14,0.03,0.42,0.18,0.06,1.08,Não,Não,Não,Não,Não,43307,Sim,49,Sim,Sim,1,11.37,Não,Não,NaN,341,1
1430,1920,2015-06-01,38,18.a.60,44.0,Parda,F,HAS,1,B,1,Sim,7.0,Sim,1,UNIFESP,16,Entre_0_50,2,31,15,39,4,0,homozigoto,heterozigoto,heterozigoto,0.23,0.00,0.00,0.19,0.06,1.14,0.42,0.08,3.36,Não,Não,Não,Não,Não,43307,Não,38,Sim,Sim,1,30.70,Não,Não,NaN,921,1
32188,39003,2001-07-30,55,18.a.60,3.0,Branca,F,Pielonefrite,1,B,0,Não,0.0,Não,0,HCFMUSP,0,Zero,24,29,27,44,4,7,heterozigoto,heterozigoto,heterozigoto,0.23,0.22,5.06,0.04,0.20,0.80,0.16,0.08,1.28,NaN,Não,NaN,NaN,Não,43307,Sim,53,Sim,Sim,1,31.73,Não,Não,NaN,952,1


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48153 entries, 0 to 48152
Data columns (total 53 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Id                          48153 non-null  int64  
 1   date                        48153 non-null  object 
 2   age_at_list_registration    48153 non-null  int64  
 3   age_cat                     48153 non-null  object 
 4   time_on_Dialysis            46817 non-null  float64
 5   race                        48153 non-null  object 
 6   sex                         48153 non-null  object 
 7   underline_disease           48153 non-null  object 
 8   diabetes                    48153 non-null  int64  
 9   Blood_type                  48153 non-null  object 
 10  number_transfusion          48153 non-null  int64  
 11  gestation                   48153 non-null  object 
 12  number_gestation            19464 non-null  float64
 13  prior_transplant            481

#### Preliminary Observations about the data:
- 12 float; 17 int; and 24 object variables
- 48153 tuples and 54 features

In [9]:
# getting percentage of missing values per feature

data.isna().sum()/len(data)*100
# remove features with missing value percentage greater than 45
# create a flag (look at features / check )

Id                             0.000000
date                           0.000000
age_at_list_registration       0.000000
age_cat                        0.000000
time_on_Dialysis               2.774490
race                           0.000000
sex                            0.000000
underline_disease              0.000000
diabetes                       0.000000
Blood_type                     0.000000
number_transfusion             0.000000
gestation                      0.000000
number_gestation              59.578842
prior_transplant               0.000000
number_prior_transplant        0.000000
subregion                      0.000000
cPRA                           0.000000
cPRA_cat                       0.000000
HLA_A1                         0.000000
HLA_A2                         0.000000
HLA_B1                         0.000000
HLA_B2                         0.000000
HLA_DR1                        0.000000
HLA_DR2                        0.000000
DR_00                          0.000000


#### Preliminary Observations about the data:
- time_on_Dialysis, number_gestation, chagas, agHBs, anti.HCV, razon_removed - these 6 features might need 'missing value imputation' We will consider it after understanding the nature of the features. 
- we also observe that the names of features don't really require a change in name. they all seem to follow basic conventions for easy manipulation. So we will call them as such, unless the need arises to change the feature name to suit a particular need, may during the feature engineering phase
- 'razon_removed' has 68% of its values missing. We may have to delete this column. But first we will check the busine use of this feature. Same is the case with 'number_gestation'.

In [ ]:
#basic data description of object class features.

data.describe(include = object)

,date,age_cat,race,sex,underline_disease,Blood_type,gestation,prior_transplant,subregion,cPRA_cat,...,anti.HBc,anti.HCV,agHBs,patient_still_on_list,death,Transplant,Transplant_Y_N,priorization,removed_list,razon_removed
count,48153,48153,48153,48153,48153,48153,48153,48153,48153,48153,...,48153,44141,44141,48153,48153,48153,48153,48153,48153,15295
unique,4108,3,4,2,5,4,2,2,4,4,...,2,2,2,2,2,3,2,1,2,10
top,2016-06-20,18.a.60,Branca,M,Outras,O,Não,Não,UNIFESP,Zero,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Removido (suspenso > 365 dias)
freq,94,34752,32455,28684,16626,23640,36122,41983,23166,34205,...,47380,43595,44022,37250,36592,25289,34421,48153,32858,12985


#### Preliminary Observations about the categorical features:
- 'date' should be converted into datetime object. 
- Portuguese terms needs to be replaced with 0 and 1. We will set Não=0 and Sim = 1. 
- 
- 

In [ ]:
#getting a hands on idea about the attributes
data.describe()

,Id,age_at_list_registration,time_on_Dialysis,diabetes,number_transfusion,number_gestation,number_prior_transplant,cPRA,HLA_A1,HLA_A2,...,calculated_frequency_B.f,calculated_frequency_A.f1,calculated_frequency_A.f2,calculated_frequency_A.f,date_acutal,Time_death,X36MthsTx,Time_Tx,time,event
count,48153.000000,48153.000000,46817.000000,48153.000000,48153.000000,19464.000000,48153.000000,48153.000000,48153.000000,48153.000000,...,48153.000000,48153.000000,48153.000000,48153.000000,48153.0,48153.000000,48153.000000,48153.000000,48153.000000,48153.000000
mean,29486.740515,48.613399,21.054254,0.792599,0.412518,2.040793,0.146325,14.402093,10.359168,28.816211,...,1.182590,0.235889,0.118824,2.697248,43307.0,75.726372,0.205117,35.527375,1090.576350,1.558864
std,16186.744194,14.707031,29.212685,0.405450,0.635550,2.413947,0.409814,29.119877,13.485239,21.826129,...,1.096077,0.145272,0.098990,2.789760,0.0,57.602143,0.403791,30.977942,955.582713,1.138825
min,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,43307.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,15902.000000,39.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,11.000000,...,0.360000,0.130000,0.060000,0.780000,43307.0,27.000000,0.000000,13.670000,414.000000,1.000000
50%,29921.000000,50.000000,10.000000,1.000000,0.000000,2.000000,0.000000,0.000000,2.000000,29.000000,...,0.840000,0.170000,0.110000,1.760000,43307.0,61.000000,0.000000,26.300000,800.000000,1.000000
75%,43387.000000,59.000000,23.000000,1.000000,1.000000,3.000000,0.000000,9.000000,23.000000,33.000000,...,1.710000,0.420000,0.130000,4.200000,43307.0,112.000000,0.000000,47.670000,1470.000000,3.000000
max,56937.000000,97.000000,180.000000,1.000000,3.000000,63.000000,5.000000,100.000000,80.000000,80.000000,...,4.410000,0.420000,0.420000,17.640000,43307.0,226.000000,1.000000,225.870000,6776.000000,3.000000


In [ ]:
# let us also have a snippet of skew values of numerical features to facilitate better obsrvation
data.skew().abs()

C:\Users\NACHIK~1\AppData\Local\Temp/ipykernel_8284/2384806396.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data.skew().abs()


Id                            0.083882
age_at_list_registration      0.482444
time_on_Dialysis              2.980401
diabetes                      1.443389
number_transfusion            1.653587
number_gestation              2.701250
number_prior_transplant       3.229051
cPRA                          1.932519
HLA_A1                        1.754463
HLA_A2                        0.663204
HLA_B1                        0.251808
HLA_B2                        0.894229
HLA_DR1                       0.518564
HLA_DR2                       0.851911
calculated_frequency_DR.f1    1.657124
calculated_frequency_DR.f2    0.859342
calculated_frequency_DR.f     0.318115
calculated_frequency_B.f1     0.035232
calculated_frequency_B.f2     0.427729
calculated_frequency_B.f      1.224001
calculated_frequency_A.f1     0.300759
calculated_frequency_A.f2     1.832411
calculated_frequency_A.f      2.026850
date_acutal                   0.000000
Time_death                    0.783563
X36MthsTx                

#### Preliminary Observations about the numerical features: 
- the skewness table shows not major skewness. But we will consider a skewness of over 1 major one taking the nature of feature in mind. 
-
- 'Id' is primary key. Nothing we can do about it. 
- 'age_at_list_registration' ranges from 0 to 97 with a mean of 50 and median of 48.6. Almost normal distribution,  
- 

In [ ]:
#getting the columns list to begin building data dictionary
data.columns

Index(['Id', 'date', 'age_at_list_registration', 'age_cat', 'time_on_Dialysis',
       'race', 'sex', 'underline_disease', 'diabetes', 'Blood_type',
       'number_transfusion', 'gestation', 'number_gestation',
       'prior_transplant', 'number_prior_transplant', 'subregion', 'cPRA',
       'cPRA_cat', 'HLA_A1', 'HLA_A2', 'HLA_B1', 'HLA_B2', 'HLA_DR1',
       'HLA_DR2', 'DR_00', 'B_00', 'A_00', 'calculated_frequency_DR.f1',
       'calculated_frequency_DR.f2', 'calculated_frequency_DR.f',
       'calculated_frequency_B.f1', 'calculated_frequency_B.f2',
       'calculated_frequency_B.f', 'calculated_frequency_A.f1',
       'calculated_frequency_A.f2', 'calculated_frequency_A.f', 'chagas',
       'anti.HBc', 'anti.HCV', 'agHBs', 'patient_still_on_list', 'date_acutal',
       'death', 'Time_death', 'Transplant', 'Transplant_Y_N', 'X36MthsTx',
       'Time_Tx', 'priorization', 'removed_list', 'razon_removed', 'time',
       'event'],
      dtype='object')

### FEATURES TABLE
#### building a data Dictionary

- The major variables used in the study by Silva and team are: 

age, sex, race, comorbidities, time on dialysis, blood group, calculated panel class I (cPRA), HLA-A, HLA-B, HLA-DR, number of blood transfusions, pregnancies, previous kidney transplants, and pre-transplant serology for Hepatitis B and C.
- The following is basic data description. Full descriptin PPT.
- 


 36  chagas                      44141 non-null  object 
 37  anti.HBc                    48153 non-null  object 
 38  anti.HCV                    44141 non-null  object 
 39  agHBs                       44141 non-null  object 
 40  patient_still_on_list       48153 non-null  object 
 41  date_acutal                 48153 non-null  int64  
 42  death                       48153 non-null  object 
 43  Time_death                  48153 non-null  int64  
 44  Transplant                  48153 non-null  object 
 45  Transplant_Y_N              48153 non-null  object 
 46  X36MthsTx                   48153 non-null  int64  
 47  Time_Tx                     48153 non-null  float64
 48  priorization                48153 non-null  object 
 49  removed_list                48153 non-null  object 
 50  razon_removed               15295 non-null  object 
 51  time                        48153 non-null  int64  
 52  event                       48153 non-null  int64  

##### Numerical Features
1.	Id                            : (int) Ids of tuples
2.	age_at_list_registration      : (int) At of the patient at the time of registration
3.	time_on_Dialysis              : (float) Time spent by patient on dialysis before transplant (in months)
4.	diabetes                      : (int) Was the patient a diabetic. 1 for yes. 0 for no. #Categorical Variable
5.	number_transfusion            : (int) No. of blood transfusions the patients needed
6.	number_gestation              : (float) 
7.	number_prior_transplant       : (int) Number of previous transplants.
8.	cPRA                          : (int) Calculated panel reactive antibody. 
9.	HLA_A1                        : (int) Human leukocyte antigen
10.	HLA_A2                        : (int) .....".......
11.	HLA_B1                        : (int) .....".......
12.	HLA_B2                        : (int) .....".......
13.	HLA_DR1                       : (int) .....".......
14.	HLA_DR2                       : (int) .....".......
15.	calculated_frequency_DR.f1    : (float)
16.	calculated_frequency_DR.f2    : (float)
17.	calculated_frequency_DR.f     : (float)
18.	calculated_frequency_B.f1     : (float)
19.	calculated_frequency_B.f2     : (float)
20.	calculated_frequency_B.f      : (float)
21.	calculated_frequency_A.f1     : (float)
22.	calculated_frequency_A.f2     : (float)
23.	calculated_frequency_A.f      : (float)
24.	date_acutal                   
25.	Time_death                    
26.	X36MthsTx                     
27.	Time_Tx                       
28.	time                          
29.	event  

#####Categorical features:

 0   Id                           int64  
 1   date                          object 
 2   age_at_list_registration    48153 non-null  int64  
 3   age_cat                     48153 non-null  object 
 4   time_on_Dialysis            46817 non-null  float64
 5   race                        48153 non-null  object 
 6   sex                         48153 non-null  object 
 7   underline_disease           48153 non-null  object 
 8   diabetes                    48153 non-null  int64  
 9   Blood_type                  48153 non-null  object 
 10  number_transfusion          48153 non-null  int64  
 11  gestation                   48153 non-null  object 
 12  number_gestation            19464 non-null  float64
 13  prior_transplant            48153 non-null  object 
 14  number_prior_transplant     48153 non-null  int64  
 15  subregion                   48153 non-null  object 
 16  cPRA                        48153 non-null  int64  
 17  cPRA_cat                    48153 non-null  object 
 18  HLA_A1                      48153 non-null  int64  
 19  HLA_A2                      48153 non-null  int64  
 20  HLA_B1                      48153 non-null  int64  
 21  HLA_B2                      48153 non-null  int64  
 22  HLA_DR1                     48153 non-null  int64  
 23  HLA_DR2                     48153 non-null  int64  
 24  DR_00                       48153 non-null  object 
 25  B_00                        48153 non-null  object 
 26  A_00                        48153 non-null  object 
 27  calculated_frequency_DR.f1  48153 non-null  float64
 28  calculated_frequency_DR.f2  48153 non-null  float64
 29  calculated_frequency_DR.f   48153 non-null  float64
 30  calculated_frequency_B.f1   48153 non-null  float64
 31  calculated_frequency_B.f2   48153 non-null  float64
 32  calculated_frequency_B.f    48153 non-null  float64
 33  calculated_frequency_A.f1   48153 non-null  float64
 34  calculated_frequency_A.f2   48153 non-null  float64
 35  calculated_frequency_A.f    48153 non-null  float64
 36  chagas                      44141 non-null  object 
 37  anti.HBc                    48153 non-null  object 
 38  anti.HCV                    44141 non-null  object 
 39  agHBs                       44141 non-null  object 
 40  patient_still_on_list       48153 non-null  object 
 41  date_acutal                 48153 non-null  int64  
 42  death                       48153 non-null  object 
 43  Time_death                  48153 non-null  int64  
 44  Transplant                  48153 non-null  object 
 45  Transplant_Y_N              48153 non-null  object 
 46  X36MthsTx                   48153 non-null  int64  
 47  Time_Tx                     48153 non-null  float64
 48  priorization                48153 non-null  object 
 49  removed_list                48153 non-null  object 
 50  razon_removed               15295 non-null  object 
 51  time                        48153 non-null  int64  
 52  event                       48153 non-null  int64  